# Basic Information

In [ ]:
ph_url = 'https://www.ratemds.com/doctor-ratings/128777/Dr-Arvind-Madaan-Charlottesville-VA.html'

In [ ]:
import cloudscraper
import requests
import pandas as pd
import re
import json
import requests
from scrapy.http import TextResponse
from pprint import pprint
import cloudscraper
import time
import os
import pandas as pd
import json
import argparse
import random


headers = {'User-Agent': 'Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36',}
# headers = {'User-Agent': 'Mozilla/5.0',}


try:
    scraper = cloudscraper.create_scraper() 
    r = scraper.get(ph_url, headers = headers)
    # load the text to scrapy-type response
    response = TextResponse(r.url, body = r.text, encoding = 'utf-8')

except:
    print('Fail...')

In [ ]:
def scrape_under_cloudflare_with_2captcha(url, 
                                          headers = headers, 
                                          min_sec = 10, 
                                          provider = None,
                                          api_key = None):
    print("Request url: {}".format(url))
    
    while True:
        scraper = cloudscraper.create_scraper(
          interpreter='nodejs',
          captcha={
            'provider': provider,
            'api_key': api_key,
          }
        )
        try:
            r = scraper.get(url, headers = headers, timeout = 10)
            response = TextResponse(r.url, body = r.text, encoding = 'utf-8')
            print("\tSucc!!!")
            break
            
        except:
            print("\tFail...")
            second = random.randrange(min_sec, min_sec + 1)
            time.sleep(second)
            
    return response

min_sec = 0
api_key = 'Your_API_Key'
provider = '2captcha'


response = scrape_under_cloudflare_with_2captcha(ph_url, 
                                                 headers = headers, 
                                                 min_sec = min_sec, 
                                                 provider = provider,
                                                 api_key = api_key)

In [ ]:
# import json
# with open('RateMD_Sample.html', 'w') as f:
#     f.write(str(response.body.decode()))

# Json Information

In [ ]:
xpath = './/script//text()'
selectors = response.xpath(xpath)

js_texts = selectors.extract()
js_texts = [i for i in js_texts]
len(js_texts)

In [ ]:
# idx = 31
# json_string = js_texts[idx]
# doctorDetailSideAdData = [i for i in json_string.split('\n') if 'window.DATA.doctorDetailSideAdData = JSON.parse(' in i][0]
# doctorDetailProps = [i for i in json_string.split('\n') if 'window.DATA.doctorDetailProps = JSON.parse(' in i][0]

In [ ]:
doc_info = {}

## Meta Info

In [ ]:
# idx = 14
json_string = [i for i in js_texts if 'aggregateRating' in i][0]
# print(json_string)
data = json.loads(json_string)


ratingCount = int(data['aggregateRating']['ratingCount'])

for col in ['address', 'image', 'name', 'telephone']:
    doc_info[col] = data[col]
    
for col in ['bestRating', 'ratingCount', 'ratingValue']:
    doc_info[col] = float(data['aggregateRating'][col])
    
doc_info

## Doctor Info

In [ ]:
# idx = 19
json_string = [i for i in js_texts if 'window.DATA.Header_props = JSON.parse(' in i][0]
string_clean = json_string.split(";\n")[1].split(' = JSON.parse(')[1][:-1]
# string_clean

x = string_clean
x = json.loads(x)
doctor_json = json.loads(x)['doctor']

pprint(doctor_json)

In [ ]:
for k, v in doctor_json.items():
    doc_info[k] = v
    
pprint(doc_info)

## Review Info

In [ ]:
ph_url

In [ ]:
ratingCount

In [ ]:
def get_reviews_from_ph_url(ph_url, ratingCount, headers, min_sec, provider,  api_key ):
    L = []
    for page in range(int((ratingCount-1) / 10) + 1):
        # print(page)
        review_url = '{}?json=true&page={}'.format(ph_url, page)


        # url = review_url
        # print(url)
        response = scrape_under_cloudflare_with_2captcha(review_url, 
                                                         headers = headers, 
                                                         min_sec = min_sec, 
                                                         provider = provider,
                                                         api_key = api_key)
        # have a look at its body
        # print(str(response.body.decode()))
        reviews = response.json()['results']
        print('Get Reviews {} from last url.\n'.format(len(reviews)))
        L = L + reviews
    return L


reviews = get_reviews_from_ph_url(ph_url, ratingCount, headers, min_sec, provider,  api_key )

In [ ]:
len(L)

# All In One

In [ ]:
import cloudscraper
import requests
import pandas as pd
import re
import json
import requests
from scrapy.http import TextResponse
from pprint import pprint
import cloudscraper
import time
import os
import pandas as pd
import json
import argparse
import random
from datetime import datetime




def scrape_under_cloudflare_with_2captcha(url, 
                                          headers = headers, 
                                          min_sec = 0, 
                                          provider = None,
                                          api_key = None):
    print("Request url: {}".format(url))
    
    while True:
        scraper = cloudscraper.create_scraper(
          interpreter='nodejs',
          captcha={
            'provider': provider,
            'api_key': api_key,
          }
        )
        try:
            r = scraper.get(url, headers = headers, timeout = 10)
            response = TextResponse(r.url, body = r.text, encoding = 'utf-8')
            print("\tSucc!!!")
            break
            
        except:
            print("\tFail...")
            second = random.randrange(min_sec, min_sec + 1)
            time.sleep(second)
            
    return response


def get_reviews_from_ph_url(ph_url, ratingCount, headers, min_sec, provider,  api_key):
    L = []
    for page in range(int((ratingCount-1) / 10) + 1):
        # print(page)
        review_url = '{}?json=true&page={}'.format(ph_url, page)


        # url = review_url
        # print(url)
        response = scrape_under_cloudflare_with_2captcha(review_url, 
                                                         headers = headers, 
                                                         min_sec = min_sec, 
                                                         provider = provider,
                                                         api_key = api_key)
        # have a look at its body
        # print(str(response.body.decode()))
        reviews = response.json()['results']
        print('\tGet Reviews {} from last url.'.format(len(reviews)))
        L = L + reviews
    return L


def get_physician_info_from_ratemd_url(ph_url, headers, min_sec, provider, api_key):

    doc_info = {}


    response = scrape_under_cloudflare_with_2captcha(ph_url, 
                                                        headers = headers, 
                                                        min_sec = min_sec, 
                                                        provider = provider,
                                                        api_key = api_key)
    
    xpath = './/script//text()'
    selectors = response.xpath(xpath)

    js_texts = selectors.extract()
    js_texts = [i for i in js_texts]

    # step 1:
    json_string = [i for i in js_texts if 'aggregateRating' in i][0]
    # print(json_string)
    data = json.loads(json_string)


    ratingCount = int(data['aggregateRating']['ratingCount'])

    for col in ['address', 'image', 'name', 'telephone']:
        doc_info[col] = data[col]
        
    for col in ['bestRating', 'ratingCount', 'ratingValue']:
        doc_info[col] = float(data['aggregateRating'][col])

    # step 2:
    # idx = 19
    json_string = [i for i in js_texts if 'window.DATA.Header_props = JSON.parse(' in i][0]
    string_clean = json_string.split(";\n")[1].split(' = JSON.parse(')[1][:-1]
    # string_clean

    x = string_clean
    x = json.loads(x)
    doctor_json = json.loads(x)['doctor']
    for k, v in doctor_json.items():
        doc_info[k] = v

    reviews = get_reviews_from_ph_url(ph_url, ratingCount, headers, min_sec, provider,  api_key )
    doc_info['reviews'] = reviews

    return doc_info


In [ ]:

    
headers = {'User-Agent': 'Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36',}
min_sec = 0
api_key = 'a0a94dedf2ad039c083871e62e3375bc'
provider = '2captcha'



ph_url = 'https://www.ratemds.com/doctor-ratings/128777/Dr-Arvind-Madaan-Charlottesville-VA.html'
doc_info = get_physician_info_from_ratemd_url(ph_url, headers, min_sec, provider, api_key)

print('Reviews: reported {} vs collected {}'.format(int(doc_info['ratingCount']), len(doc_info['reviews'])))
           